# Graph Neural Networks

In this tutorial, we will explore the implementation of graph neural networks and investigate what representations these networks learn. Along the way, we'll see how PyTorch Geometric and TensorBoardX can help us with constructing and training graph models.

# Preliminaries: PyTorch

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics

# Graph Neural Network
# We first need to load the Cora dataset:

In [11]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')



# Now let’s implement a two-layer GCN
The constructor defines two GCNConv layers which get called in the forward pass of our network. Note that the non-linearity is not integrated in the conv calls and hence needs to be applied afterwards (something which is consistent accross all operators in PyTorch Geometric). Here, we chose to use ReLU as our intermediate non-linearity between and finally output a softmax distribution over the number of classes. Let’s train this model on the train nodes for 200 epochs:

In [12]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # you can change convolutions here 
        #self.conv1 = SAGEConv(dataset.num_node_features, 16)
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

# Let’s train this model on the train nodes for 200 epochs:

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    _, pred = model(data).max(dim=1)
    correct = float (pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
    acc = correct / data.train_mask.sum().item()
    print('Epoch: %d, Accuracy: %.4f'%(epoch,acc))

Epoch: 0, Accuracy: 0.5143
Epoch: 1, Accuracy: 0.6500
Epoch: 2, Accuracy: 0.7571
Epoch: 3, Accuracy: 0.8214
Epoch: 4, Accuracy: 0.8714
Epoch: 5, Accuracy: 0.8929
Epoch: 6, Accuracy: 0.9143
Epoch: 7, Accuracy: 0.8929
Epoch: 8, Accuracy: 0.9286
Epoch: 9, Accuracy: 0.9500
Epoch: 10, Accuracy: 0.9643
Epoch: 11, Accuracy: 0.9429
Epoch: 12, Accuracy: 0.9429
Epoch: 13, Accuracy: 0.9571
Epoch: 14, Accuracy: 0.9571
Epoch: 15, Accuracy: 0.9643
Epoch: 16, Accuracy: 0.9571
Epoch: 17, Accuracy: 0.9571
Epoch: 18, Accuracy: 0.9786
Epoch: 19, Accuracy: 0.9857
Epoch: 20, Accuracy: 0.9786
Epoch: 21, Accuracy: 0.9786
Epoch: 22, Accuracy: 0.9929
Epoch: 23, Accuracy: 0.9714
Epoch: 24, Accuracy: 0.9786
Epoch: 25, Accuracy: 0.9714
Epoch: 26, Accuracy: 0.9786
Epoch: 27, Accuracy: 0.9857
Epoch: 28, Accuracy: 1.0000
Epoch: 29, Accuracy: 0.9643
Epoch: 30, Accuracy: 0.9857
Epoch: 31, Accuracy: 0.9929
Epoch: 32, Accuracy: 0.9929
Epoch: 33, Accuracy: 0.9929
Epoch: 34, Accuracy: 1.0000
Epoch: 35, Accuracy: 0.9857
Ep

# Finally we can evaluate our model on the test nodes:

In [14]:
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7540


More examples: https://github.com/rusty1s/pytorch_geometric/tree/master/examples